In [7]:
import numpy as np
# import torch
# import matplotlib.pyplot as plt
import pandas as pd  

In [8]:
data_file = '../../../data/fitRecord/processed/FitRecInterp.pickle'
# data_file = '../data/'
raw_data = np.load(data_file, allow_pickle=True)
print(raw_data['nvec'])
print(raw_data['nmod'])
train_data_list = raw_data['train_folds']
test_data_list = raw_data['test_folds']
print(train_data_list[0].shape)
print(test_data_list[0].shape)

[500, 20, 50]
3
(33379, 5)
(16599, 5)


In [12]:
fold = 0
tr_ind = train_data_list[fold][:,0:3]
tr_y = train_data_list[fold][:,-1]
tr_T = train_data_list[fold][:,-2]

te_ind =  test_data_list[fold][:,0:3]
te_y =  test_data_list[fold][:,-1]
te_T =  test_data_list[fold][:,-2]

all_data = pd.DataFrame()
all_ind = np.concatenate((tr_ind,te_ind))
all_time = np.concatenate((tr_T,te_T)) # zheng has alreadly normalized the time into 0-1
all_y = np.concatenate((tr_y,te_y))

# standardize the y
all_y = (all_y - all_y.mean()) / all_y.std()

all_data['location'] = all_ind[:,0].astype(np.int64)
all_data['pollutant'] = all_ind[:,1].astype(np.int64)
# all_data['period'] = all_ind[:,2].astype(np.int64)
all_data['y'] = all_y.astype(float)
all_data['time'] = all_time




decimal = 2

all_time = np.round(all_time,decimal)

all_data['time'] = all_time


time_uni_all = np.sort(np.unique(all_time))
time_uni_tr = np.sort(np.unique(all_time))


if len(time_uni_all)==len(time_uni_tr):
    print('no never-seen time-stamp in test data')
else: print('exist never-seen time-stamp in test data')

no never-seen time-stamp in test data


In [13]:
def unique_recoding(data,colum_name):
    # colum_name = 'movieId'
    unique_key = np.sort(data[colum_name].unique())
    recode_dict = {key:id for id,key in enumerate(unique_key)}

    new_column = data[colum_name].apply(lambda x:recode_dict[x])
    # data[colum_name] = new_column
    print('ndim of %s is %d'%(colum_name,len(new_column.unique())))
    return recode_dict, new_column

def unique_recoding_array(data_array):
    
    unique_key = np.unique(data_array)
    recode_dict = {key:id for id,key in enumerate(unique_key)}

    new_array =  np.array(list(map(lambda x:recode_dict[x], data_array)))
    # data[colum_name] = new_column
    # print('ndim of %s is %d'%(colum_name,len(new_column.unique())))
    return new_array

recode_dict_globle,all_data['timestamp_disct'] = unique_recoding(all_data,'time') 
# use for baseline, where we simply encoding the timastamp as extra mode 

ndim of time is 101


In [14]:
data_save = {}
data_save['ndims'] = raw_data['nvec']
data_save['num_node'] = np.sum(data_save['ndims'])
data_save['data'] = []
data_save['time_uni'] = np.sort(np.unique(all_time)) # sorted unique timestamps of all data
data_save['time_id_table'] = recode_dict_globle # timestamp-disct id map table

N = len(all_y)
ratio = 0.8
N_train = int(N*ratio)

for fold in range(5):

    fold_data = {}

    idx = np.random.permutation(N)
    train_idx = idx[:N_train]
    test_idx = idx[N_train:]

    tr_ind = all_ind[train_idx,:]
    tr_y = all_y[train_idx]
    tr_T = all_time[train_idx]

    te_ind =  all_ind[test_idx,:]
    te_y =  all_y[test_idx]
    te_T =  all_time[test_idx]


    # we have to sort the training data by the time-stamp
    # also sort the test data, but it's not necessary
    tr_sort_id = np.argsort(np.squeeze(tr_T))
    fold_data['tr_ind'] = tr_ind[tr_sort_id,:].astype(int)
    fold_data['tr_y'] = tr_y[tr_sort_id]
    fold_data['tr_T'] = tr_T[tr_sort_id]

    te_sort_id = np.argsort(np.squeeze(te_T))
    fold_data['te_ind'] = te_ind[te_sort_id,:].astype(int)
    fold_data['te_y'] = te_y[te_sort_id]
    fold_data['te_T'] = te_T[te_sort_id]

    # we discrete the time-stamps of all data(both train and test),and encoding them with id
    # there are two usages of this feature:
    # 1: use them as extra mode in some non-dynamict baselines  
    # 2: to efficient identify the group of obseved entries with same time-stamps

    fold_data['tr_T_disct'] = np.array(list(map(lambda x:recode_dict_globle[x], np.squeeze(fold_data['tr_T'])))).astype(int)
    fold_data['te_T_disct'] = np.array(list(map(lambda x:recode_dict_globle[x], np.squeeze(fold_data['te_T'])))).astype(int)
    data_save['data'].append(fold_data)

    
file_name = '../fitRecord_50k.npy'
np.save(file_name, data_save)

In [15]:
recode_dict_globle

{0.0: 0,
 0.01: 1,
 0.02: 2,
 0.03: 3,
 0.04: 4,
 0.05: 5,
 0.06: 6,
 0.07: 7,
 0.08: 8,
 0.09: 9,
 0.1: 10,
 0.11: 11,
 0.12: 12,
 0.13: 13,
 0.14: 14,
 0.15: 15,
 0.16: 16,
 0.17: 17,
 0.18: 18,
 0.19: 19,
 0.2: 20,
 0.21: 21,
 0.22: 22,
 0.23: 23,
 0.24: 24,
 0.25: 25,
 0.26: 26,
 0.27: 27,
 0.28: 28,
 0.29: 29,
 0.3: 30,
 0.31: 31,
 0.32: 32,
 0.33: 33,
 0.34: 34,
 0.35: 35,
 0.36: 36,
 0.37: 37,
 0.38: 38,
 0.39: 39,
 0.4: 40,
 0.41: 41,
 0.42: 42,
 0.43: 43,
 0.44: 44,
 0.45: 45,
 0.46: 46,
 0.47: 47,
 0.48: 48,
 0.49: 49,
 0.5: 50,
 0.51: 51,
 0.52: 52,
 0.53: 53,
 0.54: 54,
 0.55: 55,
 0.56: 56,
 0.57: 57,
 0.58: 58,
 0.59: 59,
 0.6: 60,
 0.61: 61,
 0.62: 62,
 0.63: 63,
 0.64: 64,
 0.65: 65,
 0.66: 66,
 0.67: 67,
 0.68: 68,
 0.69: 69,
 0.7: 70,
 0.71: 71,
 0.72: 72,
 0.73: 73,
 0.74: 74,
 0.75: 75,
 0.76: 76,
 0.77: 77,
 0.78: 78,
 0.79: 79,
 0.8: 80,
 0.81: 81,
 0.82: 82,
 0.83: 83,
 0.84: 84,
 0.85: 85,
 0.86: 86,
 0.87: 87,
 0.88: 88,
 0.89: 89,
 0.9: 90,
 0.91: 91,
 0.92: 9